<a href="https://colab.research.google.com/github/Teja143-sai/End1/blob/main/Smart_Number_Plate_Extractor_s1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install -q roboflow opencv-python-headless easyocr pandas

import os
import cv2
import pandas as pd
import easyocr
from roboflow import Roboflow
from google.colab import files


#number plate detection
plate_roboflow_api_key = "6dAQm7Z7qZHMPii7iNkD"
plate_roboflow_model_id = "plate-detection-p6kwa/2"

#vehicle type detection
vehicle_roboflow_api_key = "6dAQm7Z7qZHMPii7iNkD"
vehicle_roboflow_model_id = "auto-htksu/1"

#video path
video_path = '/content/13020022_3840_2160_30fps.mp4'


#video processing and number plates extraction
def process_video_and_extract_plates(video_path, plate_api_key, plate_model_id, vehicle_api_key, vehicle_model_id):

    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return

    #roboflow model initialization
    try:
        #plate detection model
        rf_plate = Roboflow(api_key=plate_api_key)
        project_plate = rf_plate.workspace().project(plate_model_id.split('/')[0])
        model_plate = project_plate.version(int(plate_model_id.split('/')[1])).model
        print("Roboflow plate detection model loaded successfully.")

        #vehicle type detection model
        rf_vehicle = Roboflow(api_key=vehicle_api_key)
        project_vehicle = rf_vehicle.workspace().project(vehicle_model_id.split('/')[0])
        model_vehicle = project_vehicle.version(int(vehicle_model_id.split('/')[1])).model
        print("Roboflow vehicle type detection model loaded successfully.")

    except Exception as e:
        print(f"Error initializing Roboflow model: {e}")
        return

    #initialize easyOCR
    reader = easyocr.Reader(['en'], gpu=False)
    print("EasyOCR reader initialized.")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = 2

    print(f"Video Info: FPS={fps:.2f}, Total Frames={total_frames}, Frame Interval={frame_interval}")

    detected_data = {}
    frame_count = 0

    output_image_dir = "detected_plates"
    os.makedirs(output_image_dir, exist_ok=True)
    print(f"Created directory to save plate images: {output_image_dir}")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        #process one frame per second
        if frame_count % frame_interval == 0:
            print(f"Processing frame {frame_count}/{total_frames}...")

            plate_results = model_plate.predict(frame, confidence=40, overlap=30).json()
            vehicle_results = model_vehicle.predict(frame, confidence=40, overlap=30).json()

            for plate_pred in plate_results['predictions']:
                px1 = int(plate_pred['x'] - plate_pred['width'] / 2)
                py1 = int(plate_pred['y'] - plate_pred['height'] / 2)
                px2 = int(plate_pred['x'] + plate_pred['width'] / 2)
                py2 = int(plate_pred['y'] + plate_pred['height'] / 2)

                plate_crop = frame[py1:py2, px1:px2]

                if plate_crop.size == 0:
                    continue

                #converting BGR to grayscale
                gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

                blurred_plate = cv2.GaussianBlur(gray_plate, (5, 5), 0)

                #applying Otsu's thresholding
                _, thresholded_plate = cv2.threshold(blurred_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                #using Easyocr to extract the numbers and characters from the number plate(preprocessed image)
                ocr_result = reader.readtext(thresholded_plate)


                if ocr_result:
                    plate_text = "".join(char for char in ocr_result[0][1] if char.isalnum()).upper()
                    if plate_text and len(plate_text) > 4:
                        print(f"  > Detected Plate: {plate_text}")

                        vehicle_type = "Unknown"
                        for vehicle_pred in vehicle_results['predictions']:
                            vx1 = int(vehicle_pred['x'] - vehicle_pred['width'] / 2)
                            vy1 = int(vehicle_pred['y'] - vehicle_pred['height'] / 2)
                            vx2 = int(vehicle_pred['x'] + vehicle_pred['width'] / 2)
                            vy2 = int(vehicle_pred['y'] + vehicle_pred['height'] / 2)

                            plate_center_x = (px1 + px2) / 2
                            plate_center_y = (py1 + py2) / 2

                            if vx1 < plate_center_x < vx2 and vy1 < plate_center_y < vy2:
                                vehicle_type = vehicle_pred['class']
                                break

                        if plate_text not in detected_data:
                             detected_data[plate_text] = vehicle_type
                             print(f"Associated Vehicle Type: {vehicle_type}")

                             image_filename = os.path.join(output_image_dir, f"{plate_text}_{frame_count}.jpg")
                             cv2.imwrite(image_filename, plate_crop)
                             print(f"Saved plate image: {image_filename}")


        frame_count += 1

    cap.release()
    print("\nVideo processing finished.")

    if detected_data:
        plates = list(detected_data.keys())
        vehicles = list(detected_data.values())

        df_data = {
            'Vehicle Type': vehicles,
            'Characters and Numbers in Number plate': plates
        }
        df = pd.DataFrame(df_data)

        output_filename = 'extracted_number_plates.csv'
        df.to_csv(output_filename, index=False)

        print(f"\nSuccessfully extracted {len(detected_data)} unique number plates.")
        print(f"Data saved to {output_filename}")

        print("\nExtracted Data")
        print(df)

        files.download(output_filename)
    else:
        print("\nNo unique number plates were extracted from the video.")


if __name__ == "__main__":
    if (plate_roboflow_api_key != "your_roboflow_api_key" and
        plate_roboflow_model_id != "your-plate-model-id/version" and
        vehicle_roboflow_api_key != "your_roboflow_api_key" and
        vehicle_roboflow_model_id != "your-vehicle-model-id/version"):

        if 'video_path' not in locals() or video_path is None or not os.path.exists(video_path):
            print("Video path not set or file not found. Please upload a video file.")
            uploaded = files.upload()
            if uploaded:
                video_filename = list(uploaded.keys())[0]
                video_path = f'/content/{video_filename}'
                print(f"Uploaded video file: {video_filename}")
            else:
                print("No video file uploaded. Exiting.")
                exit()
        process_video_and_extract_plates(
            video_path,
            plate_roboflow_api_key, plate_roboflow_model_id,
            vehicle_roboflow_api_key, vehicle_roboflow_model_id
        )

loading Roboflow workspace...
loading Roboflow project...
Roboflow plate detection model loaded successfully.
loading Roboflow workspace...
loading Roboflow project...


Roboflow vehicle type detection model loaded successfully.
EasyOCR reader initialized.
Video Info: FPS=30.00, Total Frames=624, Frame Interval=2
Created directory to save plate images: detected_plates
Processing frame 0/624...
Processing frame 2/624...
Processing frame 4/624...
Processing frame 6/624...
Processing frame 8/624...
Processing frame 10/624...
Processing frame 12/624...
Processing frame 14/624...
Processing frame 16/624...
Processing frame 18/624...
  > Detected Plate: IHETUI4
Associated Vehicle Type: Unknown
Saved plate image: detected_plates/IHETUI4_18.jpg
Processing frame 20/624...
  > Detected Plate: NAAUC
Associated Vehicle Type: auto
Saved plate image: detected_plates/NAAUC_20.jpg
Processing frame 22/624...
Processing frame 24/624...
  > Detected Plate: MUTLUIO
Associated Vehicle Type: auto
Saved plate image: detected_plates/MUTLUIO_24.jpg
Processing frame 26/624...
  > Detected Plate: NAATLIG
Associated Vehicle Type: auto
Saved plate image: detected_plates/NAATLIG_26

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Task
Explore using a different OCR library specifically designed for license plate recognition to potentially improve processing speed and accuracy compared to EasyOCR in the provided Python code for vehicle and license plate detection from a video.

## Research and select a license plate ocr library

### Subtask:
Identify potential Python libraries or pre-trained models specifically designed for license plate recognition (e.g., OpenALPR, PlateRecognizer, or other open-source options).


**Reasoning**:
I need to search for Python libraries or models specifically designed for license plate recognition to fulfill the subtask. I will then select a suitable one based on ease of use and potential performance.



In [30]:
import os
import shutil

output_image_dir = "detected_plates"

# Remove the directory if it exists
if os.path.exists(output_image_dir):
    shutil.rmtree(output_image_dir)
    print(f"Removed directory: {output_image_dir}")
else:
    print(f"Directory not found: {output_image_dir}")

# Clear the detected_data dictionary if it exists
if 'detected_data' in locals():
    detected_data.clear()
    print("Cleared detected_data dictionary.")
elif 'detected_data' in globals():
    detected_data.clear()
    print("Cleared detected_data dictionary.")
else:
    print("detected_data dictionary not found in locals or globals.")

# You can optionally recreate the directory if you plan to run the detection again later
# os.makedirs(output_image_dir, exist_ok=True)
# print(f"Recreated directory: {output_image_dir}")

Removed directory: detected_plates
detected_data dictionary not found in locals or globals.
